In [176]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [177]:
df = pd.read_csv('训练集.csv')

In [178]:
df

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,package,promotion_last_5years,division,salary,satisfaction_level
0,13697,0.99,3,161.39,2,0,a,0,accounting,medium,0.681
1,1142,1.00,5,226.22,6,0,b,0,marketing,low,0.876
2,7954,0.91,5,199.45,2,0,e,0,sales,medium,0.955
3,2225,0.51,3,235.14,3,0,c,0,sales,low,0.817
4,9753,0.89,3,219.91,2,0,a,0,technical,low,0.920
...,...,...,...,...,...,...,...,...,...,...,...
11994,11971,0.89,4,150.45,3,0,e,0,accounting,high,0.849
11995,14966,0.81,5,221.28,5,0,b,0,sales,low,0.836
11996,7491,0.60,4,136.36,4,1,c,0,sales,medium,0.915
11997,12680,0.84,6,268.54,4,0,b,0,IT,low,0.099


In [179]:
df.describe()

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,promotion_last_5years,satisfaction_level
count,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000
mean,7507.265855,0.715750,3.795816,202.201511,3.500875,0.145179,0.020585,0.613853
std,4328.395859,0.170733,1.229931,49.887899,1.464127,0.352296,0.141996,0.249038
min,0.000000,0.350000,2.000000,95.100000,2.000000,0.000000,0.000000,0.076000
25%,3750.500000,0.560000,3.000000,157.135000,3.000000,0.000000,0.000000,0.439000
50%,7532.000000,0.720000,4.000000,201.020000,3.000000,0.000000,0.000000,0.646000
75%,11248.500000,0.870000,5.000000,246.020000,4.000000,0.000000,0.000000,0.817000
max,14998.000000,1.000000,7.000000,313.110000,10.000000,1.000000,1.000000,1.000000


In [180]:
for col in df.columns:
    if sum((df[col]).isnull())!= 0:
        print(col)

In [181]:
X_train = df.drop(columns=['id','satisfaction_level'])

In [182]:
y_train = df['satisfaction_level']

In [183]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [184]:
X_test = pd.read_csv('测试集.csv')

In [185]:
ind = X_test['id']

In [186]:
X_test = X_test.drop(columns=['id'])

In [187]:
from sklearn.pipeline import Pipeline
from feature_engine import categorical_encoders as ce
from sklearn.preprocessing import StandardScaler

In [188]:
prepipe = Pipeline([
                    ('onehot', ce.OneHotCategoricalEncoder(variables=['package','division','salary'],drop_last=True)),
                    ('scale', StandardScaler())
])

In [189]:
prepipe.fit(X_train,y_train)

Pipeline(steps=[('onehot',
                 OneHotCategoricalEncoder(drop_last=True,
                                          variables=['package', 'division',
                                                     'salary'])),
                ('scale', StandardScaler())])

In [190]:
X_train = prepipe.transform(X_train)

In [191]:
X_test = prepipe.transform(X_test)

In [192]:
outlier = ((X_train>3)|(X_train<-3)).any(axis=1)

In [193]:
X_train = X_train[~outlier]
y_train = y_train[~outlier]

In [200]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators = 300, max_depth=5, random_state=0)

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [201]:
y_pred

array([0.71207116, 0.10557801, 0.66071346, ..., 0.66890587, 0.66890587,
       0.41216146])

In [202]:
out = pd.DataFrame({'id':ind,'satisfaction_level':y_pred}).to_csv('submission4.csv',index=False)